# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugin

from plot import vessel_planning


import matplotlib.pyplot as plt
%matplotlib inline

# setup environment
simulation_start  = datetime.datetime(2020,3,1)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())
registry = {}
keep_resources = {}

import numpy as np

In [2]:
#Set limits
loading_limit = 2.0 
installing_limit = 1.5
sailing_empty_limit = 2.5
sailing_full_limit = 1.0

#Set durations (in hours)
loading_duration = 6
installing_duration = 8

#Set sailing speeds
sailing_speed_empty = 3
sailing_speed_full = 2

#Set amount of monopiles to install
amount_MP_to_install = 20

#Set amount of monopiles that Aeolus can hold
capacity_aeolus = 2

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

TestMoveActivity =  type(
    "TestMoveActivity",
    (
        plugin.HasWeatherPluginMoveActivity,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


TestShiftActivity =  type(
    "TestShiftActivity",
    (
        plugin.HasWeatherPluginShiftAmountActivity,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
) 

In [4]:
location_from_site = shapely.geometry.Point(-2.6939566, 48.536159)  # lon, lat
location_to_site = shapely.geometry.Point(-2.662230, 49.081062)  # lon, lat

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": amount_MP_to_install,
    "level": amount_MP_to_install,
}


data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": amount_MP_to_install,
    "level": 0,
}


from_site = Site(**data_from_site)
to_site = Site(**data_to_site)


init
level: 20
completed init
init
level: 0
completed init


In [5]:
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

data_aeolus = {
    "env": my_env,
    "name": "Aeolus",
    "geometry": location_from_site,
    "capacity": capacity_aeolus,
    "compute_v": compute_v_provider(sailing_speed_empty, sailing_speed_full),
}

aeolus = TransportProcessingResource(**data_aeolus)

init
level: 0.0
completed init


In [6]:
metocean_df = pd.read_csv("../demo/metocean_format_pieter_maart_2020.csv")
metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))

metocean_df = metocean_df.sort_index()

metocean_df["DateTime"] = [datetime.datetime.strptime(dt, '%d-%m-%Y %H:%M') for dt in metocean_df["Time"]]

In [7]:
sailing_crit_full = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum": sailing_full_limit,
    "window_length":datetime.timedelta(hours=1)
})

sailing_crit_empty = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum": sailing_empty_limit,
    "window_length":datetime.timedelta(hours=1)
})

loading_crit = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":loading_limit,
    "window_length":datetime.timedelta(hours=1)
})

installing_crit = core.WorkabilityCriterion(**{
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum": installing_limit,
    "window_length":datetime.timedelta(hours=1)
})

In [8]:
single_run = [
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": aeolus,
        "destination": from_site,
        "metocean_criteria":[sailing_crit_empty],
        "metocean_df": metocean_df,
        "timestep":1,
        "postpone_start": True,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer MP",
        "registry": registry,
        "processor": aeolus,
        "origin": from_site,
        "destination": aeolus,
        "amount": 4,    
        "duration": loading_duration*3600,
        "postpone_start": True,
        "metocean_criteria":[loading_crit],
        "metocean_df": metocean_df,
        "timestep":1,
    }),
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": aeolus,
        "destination": to_site,
        "metocean_criteria":[sailing_crit_full],
        "metocean_df": metocean_df,
        "postpone_start": True,
        "timestep":1,
    }),
    TestShiftActivity(**{
        "env": my_env,
        "name": "Transfer TP",
        "registry": registry,
        "processor": aeolus,
        "origin": aeolus,
        "destination": to_site,
        "amount": 4,
        "duration": installing_duration*3600,
        "postpone_start": True,
        "metocean_criteria":[installing_crit],
        "metocean_df": metocean_df,
        "timestep":1,
    })
]

activity = model.SequentialActivity(**{
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
})

expr = [{"type":"container", "concept": to_site, "state":"full"}] 

while_data = {
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": expr, # stop iteration once to_site is full
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


check weather plugin
True
True
True
register weather plugin
check weather plugin
True
True
True
register weather plugin
check weather plugin
True
True
True
register weather plugin
check weather plugin
True
True
True
register weather plugin
while Activity keep_resources []
get_full_event : default
start get_available
start event instance None


In [9]:
my_env.run()

conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {20: <Event() object at 0xa79316fb08>}}
amount :20
put_callback - id_ default
{}
Mover_move after mover resource request
weatherPlugin start preprocess
[['2020-03-01T00:00:00.000000000' '2020-03-06T00:49:00.000000000']
 ['2020-03-06T02:26:00.000000000' '2020-03-18T20:08:00.000000000']
 ['2020-03-18T23:03:00.000000000' '2020-03-19T00:15:00.000000000']
 ['2020-03-19T04:54:00.000000000' '2020-03-20T03:51:00.000000000']
 ['2020-03-20T05:18:00.000000000' '2020-03-20T07:30:00.000000000']
 ['2020-03-20T11:23:00.000000000' '2020-03-20T15:24:00.000000000']
 ['2020-03-20T17:49:00.000000000' '2020-03-20T19:52:00.000000000']
 ['2020-03-20T21:04:00.000000000' '2020-03-31T07:13:00.000000000']
 ['2020-03-31T15:14:00.000000000' '2020-03-31T21:59:00.000000000']]
[0.0]
we have to wait for 0.0
delay processing 0.0


start : {}
{'origin.49320300-af0e-11ea-9837-001a7dda7115': 2.0, 'destination.49307c22-af0e-11ea-898e-001a7dda7115': 14.0}
destination request : {<simpy.resources.resource.Resource object at 0x000000A792D5EEC8>: <Request() object at 0xa791576d48>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000000A792D5EEC8>: <Request() object at 0xa791576d48>, <simpy.resources.resource.Resource object at 0x000000A792D4E4C8>: <Request() object at 0xa792d4ec88>}
site request : {<simpy.resources.resource.Resource object at 0x000000A792D5EEC8>: <Request() object at 0xa791576d48>, <simpy.resources.resource.Resource object at 0x000000A792D4E4C8>: <Request() object at 0xa792d4ec88>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000000A792D5EEC8>: <Request() object at 0xa791576d48>, <simpy.resources.resource.Resource object at 0x000000A792D4E4C8>: <Request() object at 0xa792d4ec88>}
after req resource 

 ['2020-03-30T15:55:00.000000000' '2020-03-30T16:47:00.000000000']]
[32379.355241]
we have to wait for 32379.355241
delay processing 32379.355241
before delay 1584279320.6447587
after delay 1584311699.9999998
_shift_amount
start _shift_amount
amount 2.0
processor start process
install default at Dumplocatie
processor process without rate
origin store before get: [{'id': 'default', 'level': 2.0, 'capacity': 2}]
start get 2.0
store_status {'id': 'default', 'level': 2.0, 'capacity': 2}
end get 2.0
get_callback - id_ <StorePut() object at 0xa792db0e08>
start get_callback
{}
origin store after get: [{'id': 'default', 'level': 0.0, 'capacity': 2}]
after get
after check_down time
destination store before put: [{'id': 'default', 'level': 18.0, 'capacity': 20}]
<FilterStoreGet() object at 0xa792db0e08>
put_callback - id_ default
{'default': {20: <Event() object at 0xa79316fb08>}}
amount :20
destination store after put: [{'id': 'default', 'level': 20.0, 'capacity': 20}]
after put
processor end p

In [12]:
objects = [aeolus]

activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

timestamps = []
logs = [o.log["Timestamp"] for o in objects]
for log in logs:
    timestamps.extend(log)

fig = vessel_planning(objects, activities, static=True, y_scale="numbers")

fig.add_scatter(
    x=[min(timestamps), max(timestamps)],
    y=[sailing_crit_empty.maximum,sailing_crit_empty.maximum], 
    mode='lines',
    name= "Sailing empty limit"
)

fig.add_scatter(
    x=[min(timestamps), max(timestamps)],
    y=[sailing_crit_full.maximum,sailing_crit_full.maximum], 
    mode='lines',
    name= "Sailing full limit"
)

fig.add_scatter(
    x=[min(timestamps), 
       max(timestamps)], 
    y=[loading_crit.maximum,loading_crit.maximum], 
    mode='lines', 
    name= "Loading limit"
)

fig.add_scatter(
    x=[min(timestamps), 
       max(timestamps)], 
    y=[installing_crit.maximum,installing_crit.maximum], 
    mode='lines', 
    name= "Installing limit"
)

fig.add_scatter(x=metocean_df["DateTime"], y=metocean_df["Hs [m]"], mode='lines', name = "Hs [m]")